Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import requests
import time

Creating base url

In [2]:
# Create url for API call.
url = 'https://api.pushshift.io/reddit/search/submission'

Getting the most recent 50 posts from lotr that have the word wizard in it. This is a good starting point to see if lotr and harry potter posts both share the word wizard. At this point, I'm not sure how I will build a model that tests if JK Rowling took any ideas from JRR Tolkien, but until I figure out how to measure that, I will start by first extracting the data.

In [3]:
url + '?subreddit=lotr&q=wizard'

'https://api.pushshift.io/reddit/search/submission?subreddit=lotr&q=wizard'

In [4]:
params = {
    'subreddit':'lotr',
    'q': 'wizard',
    'size': 50,
}

In [5]:
res = requests.get(url, params=params)

In [6]:
res.status_code

200

In [7]:
data = res.json()


In [8]:
title = []
subreddit = []
selftext = []
created_utc = []

for item in data['data']: #post furthest in time
    for key, value in item.items():
        if key == 'title':
#             print(f'key: {key}, value: {value}')
            title.append(value)
        if key == 'selftext':
#             print(f'key: {key}, value: {value}')
            selftext.append(value)
        if key == 'subreddit':
#             print(f'key: {key}, value: {value}')
            subreddit.append(value)
        if key == 'created_utc':
#             print(f'key: {key}, value: {value}\n\n\n')
            created_utc.append(value)
    

lotr_posts = {'title': title,
        'subreddit': subreddit,
        'selftext': selftext,
        'created_utc': created_utc}

lotr_posts = pd.DataFrame(lotr_posts)

In [9]:
before = lotr_posts['created_utc'].values[-1] #post furthest in time

In [10]:
params = {
    'subreddit':'lotr',
    'q': 'wizard',
    'size': 50,
    'before' : before
}

In [11]:
res = requests.get(url, params=params)

In [12]:
res.status_code

200

In [13]:
data = res.json()

Trying to get to 4000 posts!

In [14]:
dfs = []

subreddits = ['lotr','harrypotter'] 
for subreddit in subreddits:
    #set before to current time
    before = int(time.time())
    for i in range(40):
        print(subreddit, i)
        #create params: before, subreddit, size
        params = {
        'subreddit': subreddit,
        'q': 'wizard',
        'size': 50,
        'before' : before
        }    
        #use the requests to get the response
        res = requests.get(url, params=params)    
        #turn the response into JSON
        data = res.json()    
        #turn the JSON into a DataFrame
        title2 = []
        subreddit2 = []
        selftext2 = []
        created_utc2 = []

        for item in data['data']:
            for key, value in item.items():
                if key == 'title':
        #             print(f'key: {key}, value: {value}')
                    title2.append(value)
                if key == 'selftext':
        #             print(f'key: {key}, value: {value}')
                    selftext2.append(value)
                if key == 'subreddit':
        #             print(f'key: {key}, value: {value}')
                    subreddit2.append(value)
                if key == 'created_utc':
        #             print(f'key: {key}, value: {value}\n\n\n')
                    created_utc2.append(value)


        lotr_posts2 = {'title': title2,
                'subreddit': subreddit2,
                'selftext': selftext2,
                'created_utc': created_utc2}

        lotr_posts2 = pd.DataFrame(lotr_posts2)   
        #add posts DataFrame to dfs
        dfs.append(lotr_posts2)   
        #set before to be the timestamp of the last post
        before = lotr_posts['created_utc'].values[-1]
        time.sleep(3)

#concating stuff! 
pd.concat(dfs, ignore_index=True)

lotr 0
lotr 1
lotr 2
lotr 3
lotr 4


ChunkedEncodingError: ("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))

In [ ]:
corpus = pd.concat(dfs, ignore_index=True)
corpus

Saving to CSV

In [ ]:
corpus.to_csv('reddit_corpus.csv')